# Pegando os links referente aos anos

In [1]:
import requests # requisições web
import re
from bs4 import BeautifulSoup

url = "https://www.anatel.gov.br/legislacao/portarias-normativas" # coloca o site que pretende fazer web scrapping
header = {'User-Agent': "'Mozilla/5.0'"}
html = requests.get(url, headers = header)
bs_obj = BeautifulSoup(html.text,"lxml")
posicao = bs_obj.find_all('a') # você coloca a posição que ele se encontra na página

pattern = r"\/legislacao[\/index.php]*\/portarias-normativas\/\d+"
hrefs = str(posicao).split(",")

years = [] 
for href in hrefs:
  result = re.search(pattern, href)
  if result != None:
    years.append(result.group())

years_duplicadas = list(dict.fromkeys(years))

# Adicionando um préfixo
append_str = 'https://www.anatel.gov.br'
link_years_2 = [append_str + sub for sub in years_duplicadas]

link_years = []

for i in range(0,len(link_years_2)):
  if 'index.php' not in link_years_2[i]:
    link_years.append(link_years_2[i])

link_years

['https://www.anatel.gov.br/legislacao/portarias-normativas/2021',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2020',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2019',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2018',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2017',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2016',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2015',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2014',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2013',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2012',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2011',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2010',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2009',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/2008',
 'https://www.anatel.gov.br/legislacao/portarias-normativas/20

# Pegando os links presentes em todos os anos

In [2]:
url_list = link_years # coloca o site que pretende fazer web scrapping

pattern = r"\/legislacao\/portarias-normativas\/\d+\/\d+-portaria-\d+"

Revoga = []
resolucao = [] 
resolucoes_list = []

for url in url_list:

  header = {'User-Agent': "'Mozilla/5.0'"}
  html = requests.get(url, headers = header)
  bs_obj = BeautifulSoup(html.text,"lxml")
  posicao = bs_obj.find_all('a') # você coloca a posição que ele se encontra na página

  hrefs = str(posicao).split(",")


  for href in hrefs:
    result = re.search(pattern, href)
    if result != None:
      resolucao.append(result.group())
      Revoga.append(href)

  resolucao_duplicadas = list(dict.fromkeys(resolucao))

# Adicionando um préfixo
append_str = 'https://www.anatel.gov.br'
link_resolucoes = [append_str + sub for sub in resolucao_duplicadas]

# Realizando a Leitura dos arquivos via HTML

In [27]:
url_list = link_resolucoes

Texto_list = []
Titulo_list = []

for url in url_list:

  try:
    header = {'User-Agent': "'Mozilla/5.0'"}
    html = requests.get(url, headers = header)
    bs_obj = BeautifulSoup(html.text,"lxml").text
    if 'Voltar ao topo' in bs_obj:
      texto = bs_obj.replace('  ','').replace('\n','').replace('\xa0','').replace('\t','').replace('\r','').split('Voltar ao topo')[0]
      Texto_list.append('Portaria' + texto)
        
    else:
      texto = ''
      Texto_list.append(texto)

    if 'Publicado' in bs_obj:
      titulo_separado = texto.split('Publicado')[0].split('\ufeff')[0].split('Legislação')[0].replace('}','').split('Agência')[0]
      titulo = 'Portaria'+titulo_separado
      Titulo_list.append(titulo)

    else:
      titulo = ''
      Titulo_list.append(titulo)

  except:
      print("Erro no link" + ' ' + url)

# Criando o banco de Dados Brutos

In [113]:
################################################ Criando o ID

tipo = ['803']*len(Titulo_list) # Tipo de Lei

ano_lei = []

for i in range(0,len(Titulo_list)):
  
    if 'REVOGADA' in str(Titulo_list[i]):
        result = str(Titulo_list[i]).split(',')[1].split(' (')[0][4:].replace('(REVOGADA)Anatel - ','')[-4:]
        ano_lei.append(result)
    
    elif ',' not in str(Titulo_list[i]) and 'REVOGADA' not in str(Titulo_list[i]):
        
        result = Titulo_list[i].split('Anatel -')[1][20:][-4:]
        ano_lei.append(result)
        
    else:
        
        result = Titulo_list[i].split('Anatel -')[1].split(',')[1][4:].replace(' (da SCP)','')[-4:]
        ano_lei.append(result)

num_lei_1 = [] # Pegando o numero da lei
for i in range(0,len(Titulo_list)):
    result = Titulo_list[i].split('n')[2].split(',')[0].replace(' ','').replace('º','')
    num_lei_1.append(result)
    
num_lei = [] 
for i in range(0,len(num_lei_1)):
    if 'de' in str(num_lei_1[i]):
        result = num_lei_1[i].split('de')[0]
        num_lei.append(result)
    else:
        num_lei.append(num_lei_1[i].replace('o','0'))

parte1 = [i + j for i, j in zip(tipo, num_lei)] # Juntado os valores
ID = [i + j for i, j in zip(parte1, ano_lei)] 


################################################ Data_lei

Data_lei = []

for i in range(0,len(Titulo_list)):
  
    if 'REVOGADA' in str(Titulo_list[i]):
        result = str(Titulo_list[i]).split(',')[1].split(' (')[0][4:].replace('(REVOGADA)Anatel - ','')
        Data_lei.append(result)
    
    elif ',' not in str(Titulo_list[i]) and 'REVOGADA' not in str(Titulo_list[i]):
        
        result = Titulo_list[i].split('Anatel -')[1][20:]
        Data_lei.append(result)
        
    else:
        
        result = Titulo_list[i].split('Anatel -')[1].split(',')[1][4:].replace(' (da SCP)','')
        Data_lei.append(result)

################################################ Data_publicação

Data_publica = []
for i in range(0,len(Texto_list)):
  if 'Publicado' in Texto_list[i]:
    data = Texto_list[i].split('Publicado:', 1)[1].split('|Última atualização:', 1)[0][:-5]
    Data_publica.append(data)
  else:
    data = 'NA'
    Data_publica.append(data)

################################################ Revogada

Resultado_revoga = []
for i in range(0,len(Titulo_list)):
  if 'REVOGADA' in Titulo_list[i]:
    result = True
    Resultado_revoga.append(result)
  else:
    result = False
    Resultado_revoga.append(result)



import pandas as pd

# Criando um DataFrame para alocar os outputs

dados = pd.DataFrame (ID ,columns=['ID'])
dados['Texto_lei'] = Texto_list
dados['Data_lei'] = Data_lei
dados['Data_publicação'] = Data_publica
dados['Tipo_lei'] = tipo
dados['Revogada'] = Resultado_revoga
dados['Setor'] = ['Anatel']*len(Texto_list)

dados = dados[dados.ID.apply(lambda x: x.isnumeric())]


In [114]:
dados

,ID,Texto_lei,Data_lei,Data_publicação,Tipo_lei,Revogada,Setor
0,8038452021,"Portaria Anatel - Portaria nº 845, de 06 de ag...",06 de agosto de 2021,"Sexta, 06 Agosto 2021",803,False,Anatel
1,80321822021,"Portaria Anatel - Portaria nº 2182, de 28 de d...",28 de dezembro de 2021,"Terça, 28 Dezembro 2021",803,False,Anatel
2,80321152021,"Portaria Anatel - Portaria nº 2115, de 18 de n...",18 de novembro de 2021,"Terça, 23 Novembro 2021",803,False,Anatel
3,80321132021,"Portaria Anatel - Portaria nº 2113, de 18 de n...",18 de novembro de 2021,"Terça, 23 Novembro 2021",803,False,Anatel
4,80321142021,"Portaria Anatel - Portaria nº 2114, de 18 de n...",18 de novembro de 2021,"Terça, 23 Novembro 2021",803,False,Anatel
...,...,...,...,...,...,...,...
257,8031242005,"Portaria Anatel - Portaria nº 124, de 29 de ab...",29 de abril de 2005,"Sexta, 29 Abril 2005",803,True,Anatel
258,8033002003,"Portaria Anatel - Portaria nº 300, de 15 de se...",15 de setembro de 2003,"Segunda, 15 Setembro 2003",803,True,Anatel
259,8031102003,"Portaria Anatel - Portaria nº 110, de 16 de ab...",16 de abril de 2003,"Domingo, 13 Abril 2003",803,True,Anatel
260,8031042000,"Portaria Anatel - Portaria nº 104 , de 13 de a...",13 de abril de 2000,"Quarta, 03 Maio 2000",803,True,Anatel


In [117]:
dados.to_csv("Portarias_ANATEL.txt", index=False, encoding='utf-8-sig', sep = '汉')